# Delegates, Func, and Action

In this lecture, we explore **delegates**, one of the most powerful features in C#. Delegates let you treat methods as variables: you can pass them around, store them, and invoke them later. We will also look at **Func** and **Action**, two built-in delegate types that simplify common scenarios, and finish with **events**, which build on delegates to enable publish–subscribe programming.

## What is a Delegate?

A **delegate** is a type-safe object that can reference a method with a specific signature.

In practice, delegates shine when you pass them into another class or method as a **callback**. That way, the receiving class can execute the delegate without needing to know the implementation details.

**Example: Custom delegate with a callback parameter**

In [ ]:
var calc = new Calculator();
Console.WriteLine(calc.Compute(3, 4, MathService.Add));      // 7
Console.WriteLine(calc.Compute(3, 4, MathService.Multiply)); // 12

public delegate int Operation(int x, int y);

class MathService
{
    public static int Add(int a, int b) => a + b;
    public static int Multiply(int a, int b) => a * b;
}

class Calculator
{
    public int Compute(int a, int b, Operation operation)
    {
        return operation(a, b);
    }
}

Here, the `Calculator` doesn't know *how* the numbers should be combined. Instead, it relies on the delegate provided. This is a common and powerful use case for delegates.

## Delegate vs Func

You can write the same design using `Func` without declaring a custom delegate.

In [ ]:
var calc2 = new Calculator();
Console.WriteLine(calc2.Compute(10, 5, MathService.Add));      // 15
Console.WriteLine(calc2.Compute(10, 5, MathService.Multiply)); // 50
Console.WriteLine(calc2.Compute(10, 5, (x, y) => x - y));      // 5 (inline lambda)

class MathService
{
    public static int Add(int a, int b) => a + b;
    public static int Multiply(int a, int b) => a * b;
}

class Calculator
{
    public int Compute(int a, int b, Func<int, int, int> operation)
    {
        return operation(a, b);
    }
}

**So what's the difference?**

- A custom delegate (`Operation`) gives you a clear, domain-specific name. This can make APIs easier to read when the delegate plays an important role.
- `Func` (or `Action`) saves you from writing boilerplate delegate declarations. They are flexible, reusable, and cover most scenarios in modern C#.

## Func

`Func` is a built-in delegate that represents a method that returns a value. The last type parameter is the return type, and the others are the input parameters.

**Example: Flexible price calculator**

In [ ]:
var engine = new PriceEngine();

Func<decimal, decimal> halfOff = p => p / 2;
Func<decimal, decimal> addTax = p => p * 1.2m;
Func<decimal, decimal> noDiscount = p => p;
Func<decimal, decimal> iDontLikeYouPrice = p => p * 2.5m;

Console.WriteLine(engine.CalculatePrice(100, halfOff)); // 50
Console.WriteLine(engine.CalculatePrice(100, addTax));  // 120
Console.WriteLine(engine.CalculatePrice(100, noDiscount)); // 100
Console.WriteLine(engine.CalculatePrice(100, iDontLikeYouPrice)); // 250

class PriceEngine
{
    public decimal CalculatePrice(decimal basePrice, Func<decimal, decimal> strategy)
    {
        return strategy(basePrice);
    }
}

Here, `CalculatePrice` can apply *any* pricing strategy without needing to know the details.

## Action

`Action` is a built-in delegate that represents a method that **does not return a value**. Instead of hardcoding behaviour into a method, you can inject what should happen.

**Example: Logging strategy**

In [ ]:
using System.IO;

var processor = new Processor();
// Log to console
processor.Process(msg => Console.WriteLine($"Console: {msg}"));
// Log to file
processor.Process(msg => File.AppendAllText("log.txt", msg + "\n"));

class Processor
{
    public void Process(Action<string> logAction)
    {
        logAction("Processing started");
        // ... do some work ...
        logAction("Processing finished");
    }
}

This shows how `Action` makes it easy to swap out different behaviours (console vs file logging) without changing the `Processor` class.

## Events

An **event** (`EventHandler`) is a special kind of delegate used for publish–subscribe communication. An object (the publisher) can announce something has happened, and other objects (the subscribers) can respond. Miscrosoft defines the cannonical way of handling events [here](https://learn.microsoft.com/en-us/dotnet/standard/events/)

**Example**

In [ ]:
var youtuber = new YouTuber();
var listener1 = new Subscriber();
var listener2 = new Subscriber();
var listener3 = new Subscriber();
var listener4 = new Subscriber();
var listener5 = new Subscriber();

// Subscribe
// The operator += allows for a multicast Delegate, that contains a list to all the subscribers.
youtuber.Notify += listener1.OnNewVideo;
youtuber.Notify += listener2.OnNewVideoFromFavorite;
youtuber.Notify += listener3.OnNewVideo;
youtuber.Notify += listener4.OnNewVideo;
youtuber.Notify += listener5.OnNewVideoFromFavorite;

// Trigger the event
youtuber.PublishVideo("Top 10 C# Tips");

// Unsubscribe
youtuber.Notify -= listener2.OnNewVideoFromFavorite;
youtuber.Notify -= listener5.OnNewVideoFromFavorite;

// Trigger the event again
youtuber.PublishVideo("Advanced Event Handling");

// EventArgs subclass to hold event data
public class VideoEventArgs : EventArgs
{
    public string Title { get; }
    public VideoEventArgs(string title) => Title = title;
}

public class YouTuber
{
    // EventHandler is a special kind of delegate for events.
    public event EventHandler<VideoEventArgs>? Notify;

    public void PublishVideo(string title)
    {
        Console.WriteLine($"YouTuber: Publishing '{title}'...");
        OnNotify(new VideoEventArgs(title));
    }

    // Typically, to raise an event, you add a method that is marked as protected and virtual
    // This way, classes that inherit from YouTuber can override if needed!
    protected virtual void OnNotify(VideoEventArgs e)
    {
        Notify?.Invoke(this, e);
    }
}

public class Subscriber
{
    public void OnNewVideo(object? sender, VideoEventArgs e)
    {
        Console.WriteLine($"Subscriber: New video published! Title: {e.Title}");
    }

    public void OnNewVideoFromFavorite(object? sender, VideoEventArgs e)
    {
        Console.WriteLine($"Subscriber: My favorite YouTuber uploaded '{e.Title}'!");
    }
}

**Output**

```
YouTuber: Publishing 'Top 10 C# Tips'...
Subscriber: New video published! Title: Top 10 C# Tips
Subscriber: My favorite YouTuber uploaded 'Top 10 C# Tips'!
Subscriber: New video published! Title: Top 10 C# Tips
Subscriber: New video published! Title: Top 10 C# Tips
Subscriber: My favorite YouTuber uploaded 'Top 10 C# Tips'!
YouTuber: Publishing 'Advanced Event Handling'...
Subscriber: New video published! Title: Advanced Event Handling
Subscriber: New video published! Title: Advanced Event Handling
Subscriber: New video published! Title: Advanced Event Handling
```

**Why use events?**

- They enable loose coupling (publisher does not need to know about subscribers).
- They are the foundation for GUI programming, system notifications, and reactive patterns.

## Why Use Delegates, Func, Action, and Events?

- Pass behaviour into methods (functions as parameters).
- Build flexible and reusable code.
- Provide the foundation for LINQ, events, and asynchronous programming.
- Events add structured communication between objects.

## Summary Table

| Type | Returns a Value? | Parameters | Example |
| --- | --- | --- | --- |
| **Delegate** | Depends on signature | Defined by you | `public delegate int Operation(int x, int y);` |
| **Func** | Yes | Up to 16 input parameters | `Func<int, int, int> op = MathService.Add;` |
| **Action** | No | Up to 16 input parameters | `processor.Process(msg => Console.WriteLine(msg));` |
| **EventHandler or EventHandler<T>** | N/A | Subscribers define handlers | `public event EventHandler<VideoEventArgs>? Notify;` |